In [16]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
import json
import pandas as pd

In [17]:
file_path = r'C:\Users\b.ludwicki\Desktop\Input.xlsx'
df = pd.read_excel(file_path)

In [18]:
df.rename(columns={'Nazwa Operacji': 'transaction_label'}, inplace=True)
display(df)

,Data 1,Data 2,transaction_label,Kwota Płatności,Nr Operacji,Powyżej stówki
0,2024-05-14,2024-05-14,Zakup BLIK BGK w ramach transakcji online MFNo...,-158.12,9,yes
1,2024-05-15,2024-05-13,169.72 PLN UBER *EATS HELP.UBER.COM,-169.72,11,yes
2,2024-05-15,2024-05-13,DOP. VISA 421352******1509 WYPŁATA Z BANKOMATU...,-300.00,12,yes
3,2024-05-12,2024-05-12,Zakup BLIK PayPro S.A.None Pastelowa 860-198 P...,-694.39,14,yes
4,2024-05-14,2024-05-12,24.60 USD 1 USD=4.1156 PLN CHATGPT SUBSCRIPTIO...,-101.24,16,yes
...,...,...,...,...,...,...
247,2023-05-29,2023-05-27,144.48 PLN AMBRA BRANDS GRUNWALDZKI WROCLAW,-144.48,994,yes
248,2023-05-26,2023-05-26,Przelew na telefon,-150.00,997,yes
249,2023-05-27,2023-05-25,217.79 PLN ORLEN STACJA NR 555 WARSZAWA,-217.79,999,yes
250,2023-05-27,2023-05-25,127.59 PLN UBER *EATS HELP.UBER.COM,-127.59,1000,yes


In [20]:
# Select the first two rows
df_sample = df.head(2).copy()

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
api_key = os.getenv('OPEN_AI_PLAYGROUND_4o')
if not api_key:
    raise ValueError("API key not found in environment variables. Please set 'OPEN_AI_PLAYGROUND_4o'.")

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint="https://openaigpt4access.openai.azure.com/",
    api_key=api_key
)

# Function to get the OpenAI API response
def get_openai_response(question):
    try:
        print(f"Processing question: {question}")
        completion = client.chat.completions.create(
            model="gpt-4",  # Ensure this is the correct model and deployment name
            messages=[
                {
                    "role": "user",
                    "content": f"Label this with 3 comma separated words please: {question}",
                },
            ],
        )
        response = completion.choices[0].message['content']
        print(f"Received response: {response}")
        return response
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

# Apply the function to the 'transaction_label' column and store results in a new column
df_sample.loc[:, 'output_column'] = df_sample['transaction_label'].apply(get_openai_response)

# Display the DataFrame
print(df_sample)


Processing question: Zakup BLIK BGK w ramach transakcji online MFNone Swietokrzyska 1200-916 Warszawa ref:80946651481
Error occurred: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}
Processing question: 169.72 PLN UBER *EATS HELP.UBER.COM
Error occurred: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}
      Data 1     Data 2                                  transaction_label  \
0 2024-05-14 2024-05-14  Zakup BLIK BGK w ramach transakcji online MFNo...   
1 2024-05-15 2024-05-13                169.72 PLN UBER *EATS HELP.UBER.COM   

   Kwota Płatności  Nr Operacji Powyżej stówki output_column  
0          -158.12            9            yes         